In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import functools as ft

import numpy as np
import pandas as pd
import scanpy as sc

import celltrip


# Reading Files

In [ ]:
fnames = ['../data/scglue/Chen-2019-RNA.h5ad', '../data/scglue/Chen-2019-ATAC.h5ad']
partition_cols = ['cell_type', 'protocol']
adatas = celltrip.utility.processing.read_adatas(
    *fnames, partition_cols=partition_cols, on_disk=True)
dataloader = celltrip.utility.processing.PreprocessFromAnnData(
    *adatas, memory_efficient=True, partition_cols=partition_cols,
    top_variant=int(1e3), pca_dim=128)


# Formatting CSV Files

In [20]:
fname1 = '../data/MERFISH/s3_cell_by_gene.csv'  # '../data/UnionCom/MMD/s1_mapped1.txt'
fname2 = '../data/MERFISH/s3_mapped_cell_table.csv'


In [57]:
import anndata as ad

# Gene expression
fname = '../data/MERFISH/s3_cell_by_gene.csv'
df = pd.read_csv(fname, index_col=0, header=1)
df.index.name = 'sample_name'
adata = ad.AnnData(df)

# Spatial
fname = '../data/MERFISH/s3_mapped_cell_table.csv'
df = pd.read_csv(fname, index_col=0, header=0).set_index('sample_name')
df_obs = df[['layer']]
df = df[['xc_adjusted', 'yc_adjusted']]
adata = ad.AnnData(df, obs=df_obs)


/home/thema/miniconda3/envs/celltrip/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/thema/miniconda3/envs/celltrip/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
